In [7]:
from typing import List

import sage.all
from sage.combinat.combination import Combinations
from sage.graphs.digraph import DiGraph
from sage.graphs.graph_generators import graphs
from sage.graphs.digraph_generators import digraphs

In [5]:
def get_digraphs(N: int, min_edges: int, max_edges: int, O: bool = False) -> DiGraph:
    gen = graphs.nauty_geng(f'{N} -c')
    return digraphs.nauty_directg(gen, f"-e{min_edges}:{max_edges} {'-o' if O else ''}")

In [2]:
def get_orientations(g: DiGraph) -> List[DiGraph]:
    edges = set([(e[0], e[1]) for e in g.edges()])

    bis = set()
    for e in edges:
        if e[::-1] in edges and (e not in bis and e[::-1] not in bis):
            bis.add((e[0], e[1]))

    unis = edges - bis - set(map(lambda x: x[::-1], bis))


    undirected_g = g.to_undirected()
    orientations_of_undirected_g = undirected_g.orientations()

    orientations = []

    for orientation in orientations_of_undirected_g:
        orientation_edges = set([(e[0], e[1]) for e in orientation.edges()])
        if unis.issubset(orientation_edges):
            orientations.append(orientation)
    return orientations

In [3]:
def is_g_equals_k(g: DiGraph, goal: int) -> bool:
    vertices = set(g.vertices())
    solution = None
    for k in range(2, goal + 1):
        covering_set_candidates = Combinations(vertices, k)
        for candidate in covering_set_candidates:
            covered_set = set()
            vertex_pairs = Combinations(candidate, 2).list()
            for vertex_pair in vertex_pairs:

                paths = [g.shortest_path(vertex_pair[0], vertex_pair[1])]
                rev_paths = [g.shortest_path(vertex_pair[1], vertex_pair[0])]
                
                for path in rev_paths:
                    if path:
                        paths.append(list(path))

                for path in paths:
                    for vertex in path:
                        covered_set.add(vertex)
                if covered_set == vertices and k != goal:
                    return False, None
                elif covered_set == vertices:
                    solution = candidate
    if solution:
        return True, solution
    else:
        return False, None

In [10]:
N = 4
results = {}

for target_gn in range(2, N + 1):
    results[target_gn] = {}
    
    for M in range(N - 1, ((N * (N-1)//2)) + 1):
        di_graphs = get_digraphs(N, M, M)
        found = False
        
        for dig in di_graphs:
            if found:
                break
            for g in get_orientations(dig):
                if not len(g.edges()) == M:
                    break
                is_equal, gs = is_g_equals_k(g, target_gn)
                if is_equal:
                    results[target_gn][M] = g
                    found = True
                    break


In [11]:
results

{2: {3: Digraph on 4 vertices,
  4: Digraph on 4 vertices,
  5: Digraph on 4 vertices,
  6: Digraph on 4 vertices},
 3: {3: Digraph on 4 vertices,
  4: Digraph on 4 vertices,
  5: Digraph on 4 vertices,
  6: Digraph on 4 vertices},
 4: {3: Digraph on 4 vertices,
  4: Digraph on 4 vertices,
  5: Digraph on 4 vertices,
  6: Digraph on 4 vertices}}